In [1]:
from Exchange import *

In [2]:
# Interface
import exchange
env = exchange.make()
observation, info =  env.reset(return_info = True)
for _ in range(1000):
    action = policy(observation)
    observation, reward, done, info = env.step(action)
    if done:
        observation, info = env.reset(return_info = True)
env.close()

ModuleNotFoundError: No module named 'exchange'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.pyplot import figure
dir = "Data/"
filename = "AMZN_2021-04-01_34200000_57600000_message_10.csv"
Flow = pd.read_csv(dir + filename)
Book = pd.read_csv(dir+ filename)
# Flow = pd.read_csv(r"Data/AMZN_2021-04-01_34200000_57600000_message_10.csv")
# Book = pd.read_csv(r"Data/AMZN_2021-04-01_34200000_57600000_orderbook_10.csv")

/Users/kang/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
Flow

,34200.000241945,4,15080205,7,31163800,-1,null
0,34200.007415,5,0,48,31178000,1,NaN
1,34200.012480,3,15069985,48,31180100,-1,NaN
2,34200.029034,5,0,1,31178000,1,NaN
3,34200.047196,1,15122125,28,31155000,1,NaN
4,34200.131106,5,0,1,31178000,1,NaN
...,...,...,...,...,...,...,...
361119,57599.912692,3,405461561,7,31620000,-1,NaN
361120,57599.912692,1,405469365,7,31616900,-1,NaN
361121,57599.917400,5,0,7,31616400,1,NaN
361122,57599.937481,1,405470961,25,31610800,1,NaN


In [ ]:
picture_play_rate = 0.05
for index in range(Book.shape[0]):
    figure(figsize=(8, 6), dpi=80)
    price = Book.iloc[index,[i%2==0 for i in range(Book.shape[1])]]/10000.0
    quantity = Book.iloc[index,[i%2==1 for i in range(Book.shape[1])]]
    sc = pd.Series(quantity.tolist(), index = price.tolist())
    df = sc.sort_index()
    plt.step(df.index.tolist(), df.values.tolist())
    plt.hlines(0,min(df.index.tolist()),max(df.index.tolist()),color="red")
    plt.show()
    time.sleep(picture_play_rate)

In [ ]:
initial_order = (0, 3117.8, "AAPL", 20)
exchange = Exchange(*initial_order)
def slice(x):
    if x==1:
        return 'bid' 
    elif x==-1:
        return 'ask'

for index in range(Flow.shape[0]):
    List = Flow.iloc[index,:].tolist()
    if List[1]==1 and List[3]<=100 and (List[5]==1 or List[5]==-1):
        Tuple = tuple(
            (
            str(List[2]), # ticket id 
            '1', # order type, List[1]
            slice(List[5]),
            List[4]/10000.0, # price
            List[3], # quantity
            # List[0], # order time
            str(List[2]), # number
            "AMZN" # stock code
            )
            )
        order1= Order(*Tuple)
        Time = str(List[0])
        # if Tuple[2] == "ask":
        #     price = exchange.process_order_A(Time,order1)
        #     print(">>> Excuting ask order at time: ", Time)
        # elif Tuple[2] == "bid":
        price = exchange.process_order_B(Time,order1)
        print(">>> Excuting bid order at time: ", Time)
        print("Price:",price)

        bids = exchange.return_info()[2]
        asks = exchange.return_info()[3]
        Bids = pd.DataFrame(bids,columns=["price","quantity"])
        Asks = pd.DataFrame(asks,columns=["price","quantity"])
        New = pd.concat([Bids,Asks])
        New.set_index("price",inplace = True)
        plt.step(New.index.tolist(), New.values.tolist())
        plt.hlines(0,min(New.index.tolist()),max(New.index.tolist()),color="red")
        plt.show()

        time.sleep(0.05)

### The error comes from the ignore of the type3 order, which is the withdraw type.
As a result, we need to consider from the trader perspecitive.

### Pre-process the order flow to handle with the withdraw type order.

In [17]:
def handle_withdraw(Flow,buffer_len = 5):
    del_list = []
    for index in range(Flow.shape[0]):
    # index = 15 
        List = Flow.iloc[index,:].tolist()
        if List[1]==3:
            for i in range(1,min(buffer_len,index-1)):
                # print(">>>Flow.iloc[index - i,2]",Flow.iloc[index - i,2])
                # print("List[2]",List[2])
                if Flow.iloc[index - i,2] == List[2]:
                    del_list.append(index-i)
                    del_list.append(index)
                    break
            # print(">>>Index = {0}, del_list:{1}".format(index,len(del_list)))
            if index%100 ==0:
                print(">>>Index = {0}".format(index))
    return Flow.drop(del_list)

In [27]:
Flow2 =  handle_withdraw(Flow,buffer_len = 500)

>>>Index = 300
>>>Index = 1200
>>>Index = 1300
>>>Index = 1400
>>>Index = 1500
>>>Index = 1800
>>>Index = 1900
>>>Index = 2100
>>>Index = 2200
>>>Index = 2300
>>>Index = 2700
>>>Index = 3000
>>>Index = 3400
>>>Index = 3900
>>>Index = 4000
>>>Index = 4100
>>>Index = 4800
>>>Index = 4900
>>>Index = 5000
>>>Index = 5100
>>>Index = 5300
>>>Index = 5800
>>>Index = 6100
>>>Index = 6400
>>>Index = 6500
>>>Index = 7100
>>>Index = 7200
>>>Index = 7400
>>>Index = 7700
>>>Index = 8000
>>>Index = 8200
>>>Index = 8300
>>>Index = 8400
>>>Index = 8800
>>>Index = 9100
>>>Index = 9300
>>>Index = 9400
>>>Index = 9500
>>>Index = 9900
>>>Index = 10000
>>>Index = 10800
>>>Index = 11100
>>>Index = 11400
>>>Index = 11700
>>>Index = 11800


KeyboardInterrupt: 